In [2]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.optimizers import Adam

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
train_data = pd.read_csv("../input/train.tsv",sep='\t')
test_data = pd.read_csv("../input/test.tsv",sep='\t')

In [16]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [21]:
y_train = pd.DataFrame(train_data['Sentiment'])
y_train.loc[y_train['Sentiment'] < 2, 'Sentiment'] = 0
y_train.loc[y_train['Sentiment'] >= 2, 'Sentiment'] = 1
y_train = y_train.values
y_train

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [22]:
inital = train_data['Phrase'].values
inital = " ".join(inital)
allWords = np.array(inital.split(" "))
inital = None
# Create the Dictonary
def create_dict(words):
    my_dict = {}
    index = 1
    for x in range(len(words) - 1):
            try:
                if(my_dict[words[x]]):
                    pass
            except:
                my_dict[words[x]] = index
                index += 1
            finally:
                pass
    return my_dict

wordsDict = create_dict(allWords)

In [23]:
# Tokenizer
def tokenize(string,token_dictonary):
    tokenized_words = []
    arrayOfWords = string.split(" ")
    for x in range(len(arrayOfWords)):
        tokenized_words.append(token_dictonary[arrayOfWords[x]])
    return np.array(tokenized_words)

In [24]:
#tokenize(train_data['Phrase'][0],token_dictonary=wordsDict)
x_train_tokens = []
for x in range(len(train_data['Phrase'])):
    x_train_tokens.append(tokenize(train_data['Phrase'][x],wordsDict))

In [25]:
num_tokens = [len(token) for token in x_train_tokens+x_train_tokens]
num_tokens = np.array(num_tokens)
np.min(num_tokens)
print(len(wordsDict))

18227


In [26]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

21

In [27]:
np.sum(num_tokens<max_tokens)/len(num_tokens)

0.9353069332308087

In [28]:
x_train_pad = pad_sequences(x_train_tokens,maxlen=max_tokens,padding='pre',truncating='pre')

In [34]:
# Model
model = Sequential()
embedding_size = 20
model.add(Embedding(input_dim=len(wordsDict),output_dim=embedding_size,input_length=max_tokens))
model.add(GRU(units=16,return_sequences=True))
model.add(GRU(units=8,return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1,activation='sigmoid'))

In [35]:
model.compile(loss=keras.losses.binary_crossentropy,optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [38]:
model.fit(x_train_pad,y_train,validation_split=0.05,epochs=3,batch_size=64)

Train on 148257 samples, validate on 7803 samples
Epoch 1/1
148257/148257 [==============================] - 16s 107us/step - loss: 0.2268 - acc: 0.9054 - val_loss: 0.4767 - val_acc: 0.7969


In [71]:
pre = ["This is good","This is bad","This is fine","This is stupid"] 
allWords1tokenized = []
for x in range(len(pre)):
    allWords1tokenized.append(tokenize(pre[x],wordsDict))

allWords1tokenized_pad = pad_sequences(allWords1tokenized,maxlen=max_tokens,padding='pre',truncating='pre')
allWords1tokenized_pad
prediction = model.predict(allWords1tokenized_pad)
for x in range(len(pre)):
    print("Statement: " + pre[x])
    print('Positive' if prediction[x] > 0.5 else 'Negative')
    print()

Statement: This is good
Positive

Statement: This is bad
Negative

Statement: This is fine
Positive

Statement: This is stupid
Negative

